# Cross Validation Experiments

In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Activation
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [2]:
def create_model():
    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    width=32
    height=32
    depth=3
    classes=10
    model = Sequential()
    inputShape = (height, width, depth)
    
    model.add(Conv2D(32,(3,3),input_shape=inputShape,activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    
    model.add(Dense(1024, activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(512, activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    
    print("[INFO] compiling model...")
    opt = SGD(lr=0.01, decay=0.01 / 10, momentum=0.9, nesterov=True)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    return model

In [3]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
print(trainX.shape)
print(testX.shape)

#Few initial tricks with reshaping the data
#trainX=trainX.reshape(-1,3072)
#testX=testX.reshape(-1,3072)

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0


[INFO] loading CIFAR-10 data...
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
#This reshaping of the train and test labels were required to use KerasClassifier

Ytrain = (np.reshape(trainY, (np.product(trainY.shape)))).astype('float64')
Ytest = (np.reshape(testY, (np.product(testY.shape)))).astype('float64')

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY1 = lb.fit_transform(trainY)
testY1 = lb.transform(testY)

In [5]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer","dog", "frog", "horse", "ship", "truck"]

#Usage of keras wrappers for performing random sampling
model=KerasClassifier(build_fn=create_model,epochs=10,batch_size=64,verbose=1)
seed=7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results=cross_val_score(model,trainX,Ytrain,cv=kfold)
print(results.mean())

[INFO] compiling model...
Epoch 1/10
 2304/45000 [>.............................] - ETA: 231s - loss: 2.3017 - acc: 0.1102

KeyboardInterrupt: 